# Preparar el dataset y entrenar el modelo para el arbol de decisión

In [ ]:
!pip uninstall -y scikit-learn mlxtend imbalanced-learn

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install mlxtend imbalanced-learn

In [ ]:
!pip install numpy scipy

In [ ]:
!pip install xgboost 

In [ ]:
!pip install lightgbm

In [ ]:
!pip install tqdm

In [ ]:
pip install pyspark[ml]==3.5.3

In [ ]:
pip install cudf-cu11 dask-cuda

In [ ]:
!pip install ydata-profiling

In [ ]:
!pip install --upgrade Pillow

In [ ]:
!pip install shap

In [ ]:
import numpy
import scipy
import xgboost
import sklearn

print("Versión actual de scikit-learn:", sklearn.__version__)
print(f"Versión de numpy: {numpy.__version__}")
print(f"Versión de scipy: {scipy.__version__}")
print(f"Versión de XGBoost: {xgboost.__version__}")

In [ ]:
import re

In [ ]:
ruta_rules = "data/resultados/fpgrowth_rules.parquet"
ruta_diagnosticos = "data/resultados/icd-10-cm-tabular-2025.csv"

In [ ]:
from pyspark.sql import SparkSession

# Detener sesión anterior
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("Preparación dataset para entrenamiento del modelo predictivo") \
    .config("spark.master", "local[*]") \
    .config("spark.driver.memory", "20g") \
    .config("spark.executor.memory", "20g") \
    .getOrCreate()

In [ ]:
# Leer dataset de reglas en Parquet (formato optimizado para Spark)
df_rules = spark.read.parquet(ruta_rules)
print("Dataset reglas cargado correctamente.")

In [ ]:
df_rules.show()

In [ ]:
# Leer dataset de diagnósticos en CSV
df_diagnosticos = spark.read.csv(ruta_diagnosticos, header=True, inferSchema=True)
print("Dataset diagnósticos cargado correctamente.")

In [ ]:
from pyspark.sql.functions import col, explode

# Extraer los items de las columnas 'antecedent' y 'consequent'
all_antecedents = df_rules.select(explode(col("antecedent")).alias("diagnosis"))
all_consequents = df_rules.select(explode(col("consequent")).alias("diagnosis"))

In [ ]:
# Unir antecedentes y consecuentes
all_diagnoses = all_antecedents.union(all_consequents)

In [ ]:
from pyspark.sql.functions import regexp_extract

# Filtrar los valores que cumplen con el patrón ICD-10
diagnoses = all_diagnoses.filter(regexp_extract(col("diagnosis"), r'^[A-Za-z]\d{2}$', 0) != "")

In [ ]:
# Obtener los diagnósticos únicos
unique_diagnoses = diagnoses.select("diagnosis").distinct()
total_unique_diagnoses = unique_diagnoses.count()

In [ ]:
print(f"Diagnósticos únicos encontrados: {total_unique_diagnoses}")

In [ ]:
# Frecuencia de cada diagnóstico
from pyspark.sql.functions import desc, count
diagnosis_counts = diagnoses.groupBy("diagnosis").agg(count("*").alias("frequency")).orderBy(desc("frequency")) 
diagnosis_counts.show(truncate=False, n=58)

In [ ]:
diagnosis_counts.printSchema()

In [ ]:
# Convertir unique_diagnoses en un DataFrame con nombre de columna "icd_code"
df_diagnosticos_reglas = unique_diagnoses.withColumnRenamed("diagnosis", "icd_domain_code")

In [ ]:
df_diagnosticos_reglas.printSchema()

In [ ]:
df_diagnosticos_reglas.show()

In [ ]:
df_diagnosticos.printSchema()

In [ ]:
df_diagnosticos.show()

In [ ]:
# Realizar el JOIN para mantener solo las coincidencias en 'icd_code'
df_diagnosticos_final = (
    df_diagnosticos_reglas
    .join(df_diagnosticos, on="icd_domain_code", how="left")
    .select("icd_domain_code", "domain_description")  # Selecciona solo las columnas necesarias
    .distinct()  # Mantiene solo valores únicos
    .coalesce(1)
    .toPandas()
)

In [ ]:
df_diagnosticos_final.head()

In [ ]:
diagnosis_counts_desc = df_diagnosticos_final.merge(
    diagnosis_counts.toPandas(), 
    left_on="icd_domain_code", 
    right_on="diagnosis", 
    how="left"
)

In [ ]:
diagnosis_counts_desc = diagnosis_counts_desc.drop(columns=["diagnosis"])

In [ ]:
diagnosis_counts_desc.head()

In [ ]:
diagnosis_counts_desc = diagnosis_counts_desc.sort_values(by="frequency", ascending=False)

In [ ]:
df_diagnosticos_final.to_csv("data/resultados/df_diagnosticos_REGLAS.csv", index=False, encoding="utf-8")
print("Archivo guardado como data/resultados/df_diagnosticos_REGLAS.csv")

In [ ]:
diagnosis_counts_desc.to_csv("data/resultados/df_diagnosticos_REGLAS_frecuencias.csv", index=False, encoding="utf-8")
print("Archivo guardado como data/resultados/df_diagnosticos_REGLAS_frecuencias.csv")

In [ ]:
import pandas as pd

ruta_diagnosticos_reglas = "data/resultados/df_diagnosticos_REGLAS.csv"
df_diagnosticos_reglas = pd.read_csv(ruta_diagnosticos_reglas)
df_diagnosticos_reglas.head()

# Modelo diagnosticos reglas de asociacion

In [ ]:
ruta_arbol = "data/resultados/arbol_preprocesado.parquet"
ruta_diagnosticos_reglas = "data/resultados/df_diagnosticos_REGLAS.csv"

# Leer el archivos
df_procesado = pd.read_parquet(ruta_arbol)
df_diagnosticos_reglas = pd.read_csv(ruta_diagnosticos_reglas)

# Ver las primeras filas para comprobar que se cargó correctamente
df_procesado.head()

In [ ]:
print(df_procesado.columns)

## Selección de dominios de patologías más representativos

Para el decidir con que dominios ICD de patologias se trabajan para entrenar el modelo se sigue el criterio de la **Cobertura**.

Sea $D = \{ d_1, d_2, \dots, d_n \} $ el conjunto de dominios ICD, cada uno con un valor de *domain_description* y $\text{freq}(d_i)$ su frecuencia. Se tiene que:

1. **Total de casos** (suma de todas las frecuencias):
   $$
   T = \sum_{i=1}^{n} \text{freq}(d_i)
   $$

2. **Se ordenan los dominios de mayor a menor frecuencia**:
   $$
   \text{freq}(d_1) \geq \text{freq}(d_2) \geq \dots \geq \text{freq}(d_n).
   $$

3. **Se define un umbral** $ \tau \geq 1 $.  
   El subconjunto $ S(\tau) $ de bloques *seleccionados* es:
   $$
   S(\tau) = \{ d_i \mid \text{freq}(d_i) \geq \tau \}.
   $$

4. **El número de dominios seleccionados** es $|S(\tau)|$.

5. **La cobertura en términos de casos**:
   $$
   \text{Cobertura}(\tau) = \frac{\sum_{d_i \in S(\tau)} \text{freq}(d_i)}{T}.
   $$

Conforme $\tau $ aumenta, $|S(\tau)|$ tiende a disminuir (se descartan más clases de baja frecuencia), pero en general, también se descartan relativamente pocos casos si la cola de baja frecuencia es pequeña en proporción al total.

Para fijar $\tau$ se ha de definir un **porcentaje mínimo** de cobertura $\alpha$ (por ejemplo, 95%) y escoger la menor $\tau$ tal que:

$$
\text{Cobertura}(\tau) \geq \alpha.
$$

Con esto, se garantiza que se están reteniendo $\alpha \times 100\%$ de los casos totales.

Por tanto si se fija un $\alpha=99$, se tiene que $\tau$ es:

In [ ]:
grouped_df = (
    diagnosis_counts_desc
    .groupby("domain_description", as_index=False)
    .agg({"frequency": "sum"})
    .sort_values(by="frequency", ascending=False)
    .reset_index(drop=True)
)


total = grouped_df['frequency'].sum()
alpha = 0.99

# Calcular la freq acumulada y la cobertura acumulada
grouped_df['cumulative_freq'] = grouped_df['frequency'].cumsum()
grouped_df['coverage'] = grouped_df['cumulative_freq'] / total

# Localizar la fila donde se alcanza (o supera) esa cobertura
mask = grouped_df['coverage'] >= alpha
if mask.any():
    # Tomar el primer índice donde coverage >= 0.99
    idx = mask.idxmax()
    # Frecuencia de la clase en esa fila
    tau = grouped_df.loc[idx, 'frequency']
    print(f"Para cubrir al menos el {alpha*100}% de los casos, un tau de ~{tau} es apropiado.")
else:    
    print("No se alcanza esa cobertura con los datos actuales.")

Por tanto, los **dominios de patologias seleccionados** para entrenar el modelo predictivo son todos aquellos con una **frecuencia** de diagnóstico en el consecuente de las reglas mayor o igual a **27259**.

In [ ]:
grouped_df.to_csv("data/resultados/df_diagnosticos_REGLAS_agrupados_frecuencias.csv", index=False, encoding="utf-8")
print("Archivo guardado como data/resultados/df_diagnosticos_REGLAS_agrupados_frecuencias.csv")

In [ ]:
grouped_df_99 = grouped_df.query('frequency>=27259')
grouped_df_99

In [ ]:
grouped_df_99.to_csv("data/resultados/df_diagnosticos_REGLAS_agrupados_frecuencias_99.csv", index=False, encoding="utf-8")
print("Archivo guardado como data/resultados/df_diagnosticos_REGLAS_agrupados_frecuencias_99.csv")

In [ ]:
diagnosticos_seleccionados = df_diagnosticos_final.merge(grouped_df_99[["domain_description"]], 
                                    on="domain_description", 
                                    how="inner").drop_duplicates()

In [ ]:
diagnosticos_seleccionados

In [ ]:
import re
from collections import Counter

# Identificar todas las columnas de diagnósticos en df_procesado
columnas_diagnosticos = [col for col in df_procesado.columns if re.match(r"^[A-Z]\d{2}$", col)]

# Filtrar solo los diagnósticos seleccionados dentro de las columnas disponibles en df_procesado
diagnosticos_filtrados = [col for col in columnas_diagnosticos if col in diagnosticos_seleccionados["icd_domain_code"].tolist()]

# Identificar los parámetros bioquímicos (columnas que empiezan con "prueba_")
columnas_bioquimicas = [col for col in df_procesado.columns if col.startswith("prueba_")]

# Identificar otras variables explicativas (ni diagnósticos ni pruebas bioquímicas)
otras_explicativas = [
    col for col in df_procesado.columns if col not in (columnas_diagnosticos + columnas_bioquimicas)
]

# Conservar las variables anteriores, las pruebas bioquímicas y los diagnósticos filtrados
columnas_a_conservar = otras_explicativas + columnas_bioquimicas + diagnosticos_filtrados

df_procesado_filtrado = df_procesado[columnas_a_conservar]

In [ ]:
import numpy as np
from collections import defaultdict

# Crear diccionario de renombrado de columnas
icds_dominios = dict(zip(diagnosticos_seleccionados["icd_domain_code"].values, 
                         diagnosticos_seleccionados["domain_description"].values))

#  Renombrar las columnas del DataFrame
df_procesado_filtrado = df_procesado_filtrado.rename(columns=lambda x: icds_dominios.get(x, x), inplace=False)

# Identificar **únicamente** las columnas repetidas
columnas_repetidas = set([col for col in df_procesado_filtrado.columns if df_procesado_filtrado.columns.tolist().count(col) > 1])

# Crear diccionario de agrupación **solo para las repetidas**
columnas_agrupadas = defaultdict(list)
for col in columnas_repetidas:
    columnas_agrupadas[col] = [c for c in df_procesado_filtrado.columns if c == col]

df_fusionado = df_procesado_filtrado.copy()

In [ ]:
# Aplicar OR bitwise a las columnas repetidas y fusionarlas
for col, col_list in columnas_agrupadas.items():
    df_fusionado[col] = np.bitwise_or.reduce(df_procesado_filtrado[col_list].values, axis=1)

In [ ]:
df_fusionado = df_fusionado.loc[:, ~df_fusionado.columns.duplicated()]

In [ ]:
df_fusionado

In [ ]:
df_procesado_filtrado = df_fusionado

In [ ]:
len(diagnosticos_seleccionados["domain_description"].unique())

In [ ]:
# Mostrar resultados
print(f"Columnas originales: {df_procesado.shape[1]}")
print(f"Diagnósticos retenidos: {len(diagnosticos_seleccionados['domain_description'].unique())} de {len(columnas_diagnosticos)}")
print(f"Parámetros bioquímicos retenidos: {len(columnas_bioquimicas)}")
print(f"Otras variables explicativas retenidas: {len(otras_explicativas)}")
print(f"Nuevo total de columnas tras agrupar en bloques ICD: {df_procesado_filtrado.shape[1]}")

df_procesado_filtrado.head()

In [ ]:
ruta_guardado = "data/resultados/df_procesado_filtrado_para_arbol.csv"
df_procesado_filtrado.to_csv(ruta_guardado, index=False, encoding="utf-8")

print(f"Archivo guardado en: {ruta_guardado}")

## Cargar dataset para entrenamiento

In [ ]:
import pandas as pd
import re

ruta_df_procesado_filtrado = "data/resultados/df_procesado_filtrado_para_arbol.csv"

df_procesado_filtrado = pd.read_csv(ruta_df_procesado_filtrado)
df_procesado_filtrado.head()

## Análisis exploratorio de dataset procesado para entrenamiento

In [ ]:
df_procesado_filtrado.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Definir el subconjunto de columnas para analizar
columnas_seleccionadas = df_procesado_filtrado.columns[-10:]

# Contar la frecuencia de cada categoría en las columnas seleccionadas
frecuencias = df_procesado_filtrado[columnas_seleccionadas].sum().sort_values(ascending=False)

# Estilo ggplot-like
plt.style.use('ggplot')

# Configuración del gráfico
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(frecuencias.index, frecuencias.values, color="#4C72B0", alpha=0.85, edgecolor='black')

ax.set_facecolor("#F0F0F0")  # Fondo gris claro
ax.spines["top"].set_visible(False)   # Eliminar bordes superiores
ax.spines["right"].set_visible(False) # Eliminar bordes derechos

# Etiquetas y título con mejor tipografía
ax.set_xlabel("Bloques ICD (One-Hot Encoding)", fontsize=14, labelpad=10)
ax.set_ylabel("Frecuencia", fontsize=14, labelpad=10)
ax.set_title("Distribución de Frecuencias de Bloques ICD de Patologías", fontsize=16, pad=15)

# Rotación y alineación de etiquetas
plt.xticks(rotation=90, ha="right", fontsize=12)
plt.yticks(fontsize=12)

# Cuadrícula sutil en eje Y
ax.yaxis.grid(True, linestyle="--", alpha=0.6)

plt.show()

In [ ]:
# Data profiling para detectar posibles sesgos y desbalanceos
from ydata_profiling import ProfileReport

profile = ProfileReport(
    df_procesado_filtrado,
    title="Reporte EDA (minimal)",
    minimal=True
)
profile.to_file("data/resultados/EDA_informe.html")

## --------------------

## Definir, Optimizar hiperparámetros y entrenar el modelo

In [ ]:
columnas_y = columnas_seleccionadas

# Crear 'Y' con los diagnósticos
Y = df_procesado_filtrado[columnas_y]

# Crear 'X' eliminando las columnas de diagnóstico
X = df_procesado_filtrado.drop(columns=columnas_y)

print(f"Tamaño de X: {X.shape}")
print(f"Tamaño de Y: {Y.shape}")

In [ ]:
Y

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, hamming_loss, jaccard_score, roc_auc_score

In [ ]:
# Dividir en 80% entrenamiento y 20% prueba
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

### Prueba previa de uso correcto de GPU

In [ ]:
import xgboost as xgb
import numpy as np
import time

# Crear dataset de prueba
X_dummy = np.random.rand(100000, 50)
y_dummy = np.random.randint(2, size=100000)

# Configuración de CPU
cpu_model = xgb.XGBClassifier(tree_method="hist", device="cpu")

# Configuración de GPU
gpu_model = xgb.XGBClassifier(tree_method="hist", device="cuda")

# Entrenar con CPU
start_cpu = time.time()
cpu_model.fit(X_dummy, y_dummy)
end_cpu = time.time()

# Entrenar con GPU
start_gpu = time.time()
gpu_model.fit(X_dummy, y_dummy)
end_gpu = time.time()

# Resultados
print(f"Tiempo de entrenamiento en CPU: {end_cpu - start_cpu:.2f} segundos")
print(f"Tiempo de entrenamiento en GPU: {end_gpu - start_gpu:.2f} segundos")

### Hiperparámetros mediante búsqueda estocástica Halving

In [ ]:
import numpy as np
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from xgboost import XGBClassifier


# Diccionario donde almacenar el mejor modelo entrenado para cada etiqueta
best_models_per_label = {}

# Param distributions para la búsqueda
param_dist = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    "subsample": [0.5, 0.7, 0.9],
    "colsample_bytree": [0.4, 0.6, 0.8, 1.0],
    "reg_alpha": [0, 0.001, 0.01, 0.1, 1],  # Regularización L1
    "reg_lambda": [0, 0.001, 0.01, 0.1, 1]  # Regularización L2   
}

# Calcular min_resources para HalvingRandomSearch
N = len(X_train)
# Calcular número total de combinaciones
M = np.prod([len(v) for v in param_dist.values()])
# Factor de reducción
F = 3 
# Calcular número de iteraciones necesarias
K = int(np.floor(np.log(M) / np.log(F))) 
# Calcular min_resources
min_resources = max(5000, N // (F ** K))

In [ ]:
# Iterar por cada dominio ICD
for label in columnas_seleccionadas.to_list():
    print(f"\n=== Entrenando etiqueta: {label} ===")

    # Extraer y_train_label e y_test_label
    y_train_label = Y_train[label]
    y_test_label = Y_test[label]

    # Calcular scale_pos_weight específico
    positives = np.sum(y_train_label == 1)
    negatives = np.sum(y_train_label == 0)
    if positives == 0:
        # Si no hay positivos en train, no se puede entrenar (o se ignora la etiqueta)
        print(f"Etiqueta {label}: no hay positivos en train. Se omite.")
        continue

    spw = negatives / positives

    # Definir un XGBClassifier base con este scale_pos_weight
    xgb_clf = XGBClassifier(
        eval_metric='logloss',
        scale_pos_weight=spw, # Para el desbalanceo de dominios
        random_state=42,
        device="cuda",
        tree_method="hist",  
    )

    # Búsqueda de hiperparámetros con HalvingRandomSearch
    search = HalvingRandomSearchCV(
        estimator=xgb_clf,
        param_distributions=param_dist,
        factor=F,
        resource='n_samples',
        min_resources=min_resources,
        cv=3,
        scoring='f1_macro',
        random_state=42,
        verbose=1
    )
    
    

    # Entrenar
    search.fit(X_train, y_train_label)
    print("Mejores hiperparámetros:", search.best_params_)

    # Mejor modelo
    best_model = search.best_estimator_

    # Guardar el modelo en el diccionario
    best_models_per_label[label] = best_model

### Guardar submodelos binarios

In [ ]:
import joblib

joblib.dump(best_models_per_label, "data/resultados/xgb_binary_models_per_label.pkl")
print("Guardado en data/resultados/xgb_binary_models_per_label.pkl")

### Evaluar cada submodelo en test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    balanced_accuracy_score,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    average_precision_score,
    classification_report
)

# Arrays para reconstruir las predicciones globales (0/1)
Y_pred = np.zeros_like(Y_test.values)  # shape: (num_samples_test, num_labels)

# Arrays para reconstruir las probabilidades
Y_proba = np.zeros_like(Y_test.values, dtype=float)

# Lista para guardar las métricas de cada modelo/etiqueta
metricas_list = []

for i, label in enumerate(columnas_seleccionadas.to_list()):
    if label not in best_models_per_label:
        # Se omitió la etiqueta si no había positivos en train
        continue

    model_label = best_models_per_label[label]

    # Predicciones 0/1
    Y_pred_label = model_label.predict(X_test)  # array con 0 ó 1
    Y_pred[:, i] = Y_pred_label

    # Probabilidades de la clase 1 (columna 1 de predict_proba)
    y_proba_label = model_label.predict_proba(X_test)[:, 1]
    Y_proba[:, i] = y_proba_label

    ### Cálculo de métricas
    # Datos reales de test
    y_true_label = Y_test[label].values

    # Balanced Accuracy
    bal_acc = balanced_accuracy_score(y_true_label, Y_pred_label)

    # F1 (para la clase 1)
    f1_lbl = f1_score(y_true_label, Y_pred_label, average='binary', zero_division=0)

    # ROC-AUC
    roc_auc = roc_auc_score(y_true_label, y_proba_label)

    # Precision-Recall AUC (equivalente a average_precision_score)
    pr_auc = average_precision_score(y_true_label, y_proba_label)

    # Accuracy “clásica” (opcional)
    acc_label = accuracy_score(y_true_label, Y_pred_label)

    ### Imprimir métricas
    print(f"\n=== Etiqueta: {label} ===")
    print(f"   Balanced Accuracy: {bal_acc:.4f}")
    print(f"   F1 score (binary): {f1_lbl:.4f}")
    print(f"   ROC AUC: {roc_auc:.4f}")
    print(f"   PR AUC: {pr_auc:.4f}")
    print(f"   Accuracy (clásica): {acc_label:.4f}")

    # Imprimir un classification_report:
    print(classification_report(y_true_label, Y_pred_label, zero_division=0))

    ### Guardar métricas en nuestra lista
    metricas_list.append({
        'etiqueta': label,
        'balanced_accuracy': bal_acc,
        'f1_score': f1_lbl,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'accuracy': acc_label
    })

    ### Gráfica ROC
    fpr, tpr, _ = roc_curve(y_true_label, y_proba_label)
    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f"ROC (AUC={roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], 'r--', label="Random")
    plt.title(f"ROC Curve - {label}")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="best")
    plt.show()

    ### Gráfica Precision-Recall
    precision, recall, _ = precision_recall_curve(y_true_label, y_proba_label)
    plt.figure(figsize=(6, 4))
    plt.plot(recall, precision, label=f"PR (AUC={pr_auc:.2f})")
    plt.title(f"Precision-Recall Curve - {label}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend(loc="best")
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve

labels = columnas_seleccionadas.to_list()  # tus etiquetas (ej. 10 o 15)
n_labels = len(labels)

fig, axs = plt.subplots(n_labels, 2, figsize=(10, 5*n_labels))
# axs[i, 0] => gráfico ROC de la etiqueta i
# axs[i, 1] => gráfico PR de la etiqueta i

for i, label in enumerate(labels):
    # Si omitiste la etiqueta en el entrenamiento, sigue
    if label not in best_models_per_label:
        continue

    # Modelo y datos
    model_label = best_models_per_label[label]
    y_true = Y_test[label].values
    y_proba = model_label.predict_proba(X_test)[:, 1]

    # Calcular datos ROC
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = roc_auc_score(y_true, y_proba)

    # Calcular datos PR
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_auc = average_precision_score(y_true, y_proba)

    # --- Gráfico ROC ---
    axs[i, 0].plot(fpr, tpr, label=f"ROC (AUC={roc_auc:.2f})")
    axs[i, 0].plot([0,1],[0,1],'r--', label="Random")
    axs[i, 0].set_title(f"ROC - {label}")
    axs[i, 0].set_xlabel("False Positive Rate")
    axs[i, 0].set_ylabel("True Positive Rate")
    axs[i, 0].legend(loc="best")

    # --- Gráfico PR ---
    axs[i, 1].plot(recall, precision, label=f"PR (AUC={pr_auc:.2f})")
    axs[i, 1].set_title(f"Precision-Recall - {label}")
    axs[i, 1].set_xlabel("Recall")
    axs[i, 1].set_ylabel("Precision")
    axs[i, 1].legend(loc="best")

plt.tight_layout()
plt.show()

### Resumen de modelos con sus métricas

In [ ]:
df_metricas_styled = (
    df_metricas
    .style
    .format(precision=4)
    .hide(axis="index")
    .background_gradient(cmap='Greens', subset=['balanced_accuracy','f1_score','roc_auc','pr_auc','accuracy'])
    .set_caption("Métricas por Etiqueta (bloque ICD de patologías)")
)

df_metricas_styled

### Cargar modelo, verificar predicción de probabilidades y explicabilidad del modelo (valores SHAP)

In [ ]:
loaded_models = joblib.load("data/resultados/xgb_binary_models_per_label.pkl") 

In [ ]:
def predict_multi_label_proba(loaded_models, X_new):
    """
    Devuelve un diccionario donde cada clave es una descripcion de bloque ICD y el valor la probabilidad del bloque ICD de patologia.

    """
    predictions_proba = {}
    for label, model in loaded_models.items():
        # predict_proba => shape (n_samples, 2), tomamos la col. 1
        y_pred_proba_label = model.predict_proba(X_new)[:, 1]
        predictions_proba[label] = y_pred_proba_label
    return predictions_proba

In [ ]:
def get_feature_importance(loaded_models):
    """
    Para cada modelo XGBoost, extrae la importancia global de features,
    usando get_booster().get_score(importance_type='gain').
    Retorna un dict: { label: dict_importancia }.
    """
    feature_importance_dict = {}
    for label, model in loaded_models.items():
        booster = model.get_booster()
        importance_dict = booster.get_score(importance_type="gain")
        feature_importance_dict[label] = importance_dict
    return feature_importance_dict

In [ ]:
import shap

def get_shap_values_with_features(loaded_models, X_new):
    """
    Calcula los SHAP values para cada modelo (etiqueta) y retorna:
        {
          etiqueta: [
            { "feature_1": shap_value_1, "feature_2": shap_value_2, ... },  # para la fila 0
            { ... },                                                        # para la fila 1
            ...
          ],
          ...
        }
    """
    shap_dict = {}
    feature_names = list(X_new.columns)

    for label, model in loaded_models.items():
        explainer = shap.TreeExplainer(model)
        # shap_values => array shape (N, num_features)
        shap_values = explainer.shap_values(X_new)

        # Convertir cada fila (i) en un dict { feature_name: shap_value }
        shap_rows = []
        for i in range(shap_values.shape[0]):
            row_dict = {}
            for j, col_name in enumerate(feature_names):
                # Convertir a float pura si deseas evitar tipos float32
                row_dict[col_name] = float(shap_values[i, j])
            shap_rows.append(row_dict)

        shap_dict[label] = shap_rows

    return shap_dict

In [ ]:
X_test.iloc[[99]]

In [ ]:
paciente_ejemplo = X_test.iloc[[99]] 

In [ ]:
# Probabilídades
patology_probabilities = predict_multi_label_proba(loaded_models, paciente_ejemplo)

In [ ]:
# Importancia global
feature_imp = get_feature_importance(loaded_models)

In [ ]:
# Valores SHAP (explicación local)
shap_dict = get_shap_values_with_features(loaded_models, paciente_ejemplo)

In [ ]:
import json

# Convertir los valores del diccionario a listas
patology_probabilities_json = {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in patology_probabilities.items()}

In [ ]:
print("=== PROBABILIDADES ===")
# Imprimir el diccionario en formato JSON
print(json.dumps(patology_probabilities_json, indent=4))

In [ ]:
feature_importance_json = {}
for label, imp_dict in feature_imp.items():
    new_imp = {k: float(v) for k, v in imp_dict.items()}
    feature_importance_json[label] = new_imp
    
print("=== FEATURE IMPORTANCE ===")
print(json.dumps(feature_importance_json, indent=4))

In [ ]:
shap_values_json = {
    label: mat.tolist() if isinstance(mat, np.ndarray) else mat
    for label, mat in shap_dict.items()
}

print("=== SHAP VALUES ===")
print(json.dumps(shap_values_json, indent=4))